In [3]:
import pandas as pd
import numpy as np
import scipy.stats

gams_python_path="/Library/Frameworks/GAMS.framwork/Resources/apifiles/Python/gams"

In [4]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
output=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
cps=pd.read_stata('/Users/hannahkamen/Downloads/acs5yr_0610.dta')



In [5]:
len(cps)

15057480

In [6]:
lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')


lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

In [7]:
###read in model industry pairings in final data
gams_dta=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d2.csv')


In [8]:
####gams labor endowment data need to check sums with sector demand
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')

In [9]:
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

In [10]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [11]:
###read in dropped 

In [12]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [13]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [14]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [15]:
test=cps_codes.merge(merge_f,on='cps_code',how='outer',indicator=True)

In [16]:
cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')




In [17]:
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

In [18]:
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

In [19]:
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
                           
                           

In [20]:
# cps_m['hh']=np.where(cps_m['incwage']<=25000,'hh1','')
# cps_m['hh']=np.where(((cps_m['incwage']>25000) & (cps_m['incwage']<50000)),'hh2',cps_m['hh'])
# cps_m['hh']=np.where(((cps_m['incwage']>=50000) & (cps_m['incwage']<75000)),'hh3',cps_m['hh'])
# cps_m['hh']=np.where(((cps_m['incwage']>=75000) & (cps_m['incwage']<150000)),'hh4',cps_m['hh'])
# cps_m['hh']=np.where((cps_m['incwage']>=150000),'hh5',cps_m['hh'])
 

In [21]:
cps_m['hh']=np.where(cps_m['inctot']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['inctot']>25000) & (cps_m['inctot']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=50000) & (cps_m['inctot']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=75000) & (cps_m['inctot']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['inctot']>=150000),'hh5',cps_m['hh'])

#### get shares by all breakouts, le0(r,q,h,sk)

In [22]:
#### group data by r,q,h,s
gams_rqhs0=cps_m.reset_index()[['state_residence','state_work','hh','educ_new','incwage']].groupby(['state_residence','state_work','educ_new','hh'],as_index=False).agg({'incwage':sum})

#rename variables and limit dataframe
gams_rqhs=gams_rqhs0.rename(columns={'state_residence':'r','state_work':'q','hh':'h','educ_new':'sk'})
gams_rqhs=gams_rqhs[['r','q','h','sk','incwage']]

###classify census regions
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

###get census region lived in, census region worked in

gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(midwest),'midwest','')
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(south),'south',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(west),'west',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(northeast),'northeast',gams_rqhs['census_region_r'])


gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(midwest),'midwest','')
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(south),'south',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(west),'west',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(northeast),'northeast',gams_rqhs['census_region_q'])

####limit to where person works in same census region that they live in
gams_rqhs_lm=gams_rqhs[(gams_rqhs['census_region_r']==gams_rqhs['census_region_q'])]

###get total income by r,q,h then get income by r,q,h,s, calculate to get share of worker type income
### by region lived in worked in and income level
gams_rqhs_lm_gr=gams_rqhs_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
gams_rqhs_lm_gr=gams_rqhs_lm_gr.rename(columns={'incwage':'incwage_total'})
gams_rqhs_lm1=gams_rqhs_lm.merge(gams_rqhs_lm_gr, on=['r','q','h'],how='left')
gams_rqhs_lm1['skl_shr']=gams_rqhs_lm1['incwage']/gams_rqhs_lm1['incwage_total']


#### Deal with missing combinations in CPS data for r,q,h,sk

In [ ]:
##prepare gams table for merge
gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})[['r','q','h','sk','pop']]
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(gams_dta_le0,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])




###limit dataframe

final_shrs0=final_shrs0[['r','q','h','sk','skill_shr','combo','pop','census_share']]

#chksum to see that all proportions add to 1
check_sum0=final_shrs0.groupby(['r','q','h'],as_index=False).agg({'skill_shr':sum})
check_sum0=check_sum0[check_sum0['skill_shr']!=1]


#chksum to see that no zeros exist
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]

zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['h']


#replace 0s with census values

final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])



#check zeros agains
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]

final_shrs0_f=final_shrs0[['r','q','h','sk','skill_shr']]

In [ ]:
final_shrs0[final_shrs0['combo'].isin(missing_combos0)]['pop'].sum()

In [ ]:
469.6733563736173/9215

In [ ]:
len(final_shrs0)

In [ ]:
missing_combos0